# 0 - Import 

In [1]:
%load_ext autoreload
%autoreload 2

import time
import random
import os

from seq2seq import plot_loss
from bart import BartLayer, generate_data_loader, train, evaluate

# 1 - Train Hyperparameters

In [2]:
SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

root_path = ''

if torch.cuda.is_available():
    device = torch.device('cuda')
else : 
    device = torch.device('cpu')
model = BartLayer('sshleifer/distilbart-xsum-12-3').to(device)

tokenizer = BartTokenizerFast.from_pretrained('sshleifer/distilbart-xsum-12-3')
batch_size = 1
train_dataset, train_data_loader, dev_dataset, dev_data_loader, test_dataset, test_data_loader = generate_data_loader(batch_size,
                                                                                                                      root_path + './data/preprocess/train.pkl', 
                                                                                                                      root_path + './data/preprocess/dev.pkl',
                                                                                                                      root_path + './data/preprocess/test.pkl', 
                                                                                                                      tokenizer)

In [3]:
epochs = 8
accumulation_steps = 30
optimizer = torch.optim.Adam(model.parameters(), lr=3e-05, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
# scheduler = 

train_loss_set = []
dev_loss_set = []
dev_bleu_set = []
for epoch in range(epochs) : 
    print(f'------------------------ \n Eopchs {epoch}')
    start_time = time.time()

    train_loss = train(model, device, batch_size, accumulation_steps, train_dataset, train_data_loader, optimizer)
    dev_loss, dev_bleu  = evaluate(model, device, dev_data_loader, tokenizer)

    train_loss_set.append(train_loss)
    dev_loss_set.append(dev_loss)
    dev_bleu_set.append(dev_bleu)
    end_time = time.time()

    print(f'Epoch took : {end_time-start_time}')
plot_loss(train_loss_set, dev_loss_set, dev_bleu_set)

------------------------ 
 Eopchs 0


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 4.00 GiB total capacity; 1.33 GiB already allocated; 7.52 MiB free; 1.33 GiB reserved in total by PyTorch)